
# <span style="color:red">INSTANTIATION OF THE LLM MODEL AND THE EMBEDDING</span>

In [ ]:
import langchain_community
import os

from langchain.chat_models import ChatOpenAI

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
LANGCHAIN_TRACING_V2 = os.environ["LANGCHAIN_TRACING_V2"]
LANGCHAIN_API_KEY = os.environ["LANGCHAIN_API_KEY"]

chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model='gpt-3.5-turbo',
    temperature = 0,
    streaming = True
)


In [ ]:
### EMBEDDINGS
from langchain.embeddings.openai import OpenAIEmbeddings
## Embedding Techinque of OPENAI
embed_model=OpenAIEmbeddings(model="text-embedding-3-large")
print(len(embed_model.embed_query('hola')))

In [11]:
## READ THE DIRECTORY AND LOAD THE FILE
from langchain.document_loaders import PyPDFDirectoryLoader
# read documents
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents
dir=r'C:\Users\Aulamultimedia\Documents\practicas\eguins2\pdf_tesis/'
doc=read_doc(dir)

dir_cv=r'C:\Users\Aulamultimedia\Documents\practicas\eguins2\pdf_cv/'
doc_cv=read_doc(dir_cv)

total=doc+doc_cv



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

documents=chunk_data(docs=total,chunk_size=3000, chunk_overlap=50)
# documents_cv=chunk_data(docs=doc_cv,chunk_size=3000, chunk_overlap=50)

type(documents)


# <span style="color:red">LOAD THE DOCUMENTS AND VECTORS TO PINESTORE DB</span>

In [ ]:
## CONNECT WITH PINECONE DATABASE
from pinecone import Pinecone, ServerlessSpec
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")

#Connect to DB Pinecone
pc=Pinecone(api_key=PINECONE_API_KEY)
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)
index_name = 'eguins'

if index_name in pc.list_indexes().names():
  pc.delete_index(index_name)
  print("index {} borrado".format(index_name))

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    print("index creado con el nombre: {}".format(index_name))
    pc.create_index(
        index_name,
        dimension=3072,  # dimensionality of text-embedding models/embedding-001
        metric='cosine',
        spec=spec
        )
else:
    print("el index con el nombre {} ya estaba creado".format(index_name))

In [ ]:
## UPSERT THE VECTORS IN TO THE PINECONE DATABASE

import time
from langchain_pinecone import PineconeVectorStore
namespace = "espacio"

docsearch = PineconeVectorStore.from_documents(
    documents=documents,
    index_name=index_name,
    embedding=embed_model, 
    namespace=namespace
)
print("upserted values to {} index".format(index_name))

time.sleep(1)




# <span style="color:red">RETRIEVE AND SEARCH INTO THE CREATED PINECONE DATABASES</span>

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
pc=Pinecone(api_key=PINECONE_API_KEY)
index_name = 'eguins'
index_name_cv = 'eguinscv'
namespace = "espacio"


In [6]:
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embed_model,
    namespace=namespace,
)
retriever=vectorstore.as_retriever()

vectorstore_cv = PineconeVectorStore(
    index_name=index_name_cv,
    embedding=embed_model,
    namespace=namespace,
)
retriever_cv=vectorstore_cv.as_retriever()

In [ ]:
query = "in which companies did ezequiel used to work"
vectorstore.similarity_search(query, k=1)

In [9]:
from langchain.chains import RetrievalQA  

query = "does Ezequiel have any hands-on experience"


qa = RetrievalQA.from_chain_type(  
    llm=chat,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  

qa_cv = RetrievalQA.from_chain_type(  
    llm=chat,  
    chain_type="stuff",  
    retriever=vectorstore_cv.as_retriever()  
) 
result = qa_cv.invoke(query)

print(result['result'])

In [ ]:
print(result)

In [ ]:
empieza lo de aida

In [ ]:
# 1- Cargar API KEY de PINECONE

from dotenv import load_dotenv
import os

# Cargar las variables del archivo .env
load_dotenv()

# Acceder a las claves
PINECONE_API_KEY= os.getenv("BD_KEY")



In [2]:
# 2- Impmortar las librerias necesarias
from pinecone import Pinecone,ServerlessSpec,Index
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer




c:\Users\anben\Desktop\posgrado\LLM\llm_env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# 3- Cargar el archivo PDF
pdf_location = r'C:\Users\anben\Desktop\resume_Aida Benito.pdf'
loader = PyPDFLoader(pdf_location)
documents = loader.load()

# Dividir el documento en fragmentos pequeños
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(documents)

# Ver los primeros fragmentos para asegurar que la división fue correcta
print(f"Primeros 2 fragmentos: {split_documents[:2]}")



Primeros 2 fragmentos: [Document(page_content='AÍDABENITO\nanbenito@yahoo.com.ar\nQuilmes, BuenosAires, Argentina\nCell: (54911)6-497-6858-Skype: anbenito1975@gmail.com\nLinkedIn:https://www.linkedin.com/in/aida-benito/\nINFORMATIONSYSTEMSENGINEER\nExperiencedDataEngineer andBusiness IntelligenceConsultant. Possessesadvancedknowledgeof\ndatabases and various BI tools, as well as knowledge of programming languages, AWS and GCP\nEcosystems. Has strong communication skills. Passionate about Data Science, BigData, Internet of', metadata={'source': 'C:\\Users\\anben\\Desktop\\resume_Aida Benito.pdf', 'page': 0}), Document(page_content='Things, Artificial Intelligence, research and learning. Able to handle several projects simultaneously.\nHaspassedtheFirst CertificateinEnglishexam.\nCorecompetenciesinclude:\nFlexibility-Leadership-Teamwork-Conflict resolution-Analytical thinking-Decisionmaking\nCustomerService-Creativethinking\nPROFESSIONALEXPERIENCE\nMPSGroup-Client: DeltaAirlines\n● Desig

In [5]:
# 4- Inicializar Pinecone
from pinecone import ServerlessSpec
import time
pc=Pinecone(api_key=PINECONE_API_KEY)
cloud = 'aws'
region = 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)



index_name = "resume-index"
if index_name in pc.list_indexes().names():
  pc.delete_index(index_name)
  print("index {} borrado".format(index_name))

time.sleep(1)

# Si el índice no existe, lo creamos
if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=384,  
        metric='cosine',
        spec=spec)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()


index resume-index borrado


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [8]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings usando el modelo 
texts = [doc.page_content for doc in split_documents]
embeddings = embed_model.encode(texts)

# Preparar los datos para subirlos a Pinecone
#upsert_data = [(str(i), embeddings[i], {}) for i in range(len(embeddings))]

# Subir los embeddings al índice
#index.upsert(vectors=upsert_data)
upsert_data_generator = [
    {
        'id': str(i), 
        'values': embedding, 
        'metadata': {'text': texts[i]}  # Los metadatos (por ejemplo, texto del documento)
    }
    for i, embedding in enumerate(embeddings)
]

index.upsert(vectors=upsert_data_generator)

print(f"Embeddings generados y almacenados en el índice '{index_name}' de Pinecone.")

Embeddings generados y almacenados en el índice 'resume-index' de Pinecone.
